In [763]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, os.path

from sklearn.preprocessing import KBinsDiscretizer

%matplotlib inline

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [779]:
text_floor_replace = {
                     'подвал, 1': 1, 
                     'подвал': -1, 
                     'цоколь, 1': 1, 
                     '1,2,антресоль': 2, 
                     'цоколь': 0, 
                     'тех.этаж (6)': 6, 
                     'Подвал': -1, 
                     'Цоколь': 0, 
                     'фактически на уровне 1 этажа': 1, 
                     '1,2,3': 3, 
                     '1, подвал': 1, 
                     '1,2,3,4': 4, 
                     '1,2': 2, 
                     '1,2,3,4,5': 5, 
                     '5, мансарда': 5, 
                     '1-й, подвал': 1, 
                     '1, подвал, антресоль': 1, 
                     'мезонин': 2, 
                     'подвал, 1-3': 3, 
                     '1 (Цокольный этаж)': 0, 
                     '3, Мансарда (4 эт)': 4, 
                     'подвал,1': 1, 
                     '1, антресоль': 1, 
                     '1-3': 3, 
                     'мансарда (4эт)': 4, 
                     '1, 2.': 2, 
                     'подвал , 1 ': 1, 
                     '1, 2': 2, 
                     'подвал, 1,2,3': 3, 
                     '1 + подвал (без отделки)': 1, 
                     'мансарда': 1, 
                     '2,3': 3, 
                     '4, 5': 5, 
                     '1-й, 2-й': 2, 
                     '1 этаж, подвал': 1, 
                     '1, цоколь': 1, 
                     'подвал, 1-7, техэтаж': 7, 
                     '3 (антресоль)': 3, 
                     '1, 2, 3': 3, 
                     'Цоколь, 1,2(мансарда)': 2, 
                     'подвал, 3. 4 этаж': 4, 
                     'подвал, 1-4 этаж': 4, 
                     'подва, 1.2 этаж': 4, 
                     '2, 3': 3, 
                     '7,8': 8, 
                     '1 этаж': 1, 
                     '1-й': 1, 
                     '3 этаж': 3, 
                     '4 этаж': 4, 
                     '5 этаж': 5, 
                     'подвал,1,2,3,4,5': 5, 
                     'подвал, цоколь, 1 этаж': 1, 
                     '3, мансарда': 3,
                    ' 1, 2, Антресоль': 2,
                    ' 1-2, подвальный': 2,
                    '1 (по док-м цоколь)': 1,
                    '1, 2 этаж': 2,
                    '1, 2, 3, мансардный': 3,
                    '1,2 ': 2,
                    '1,2,3 этаж, подвал': 3,
                    '1,2,3, антресоль, технический этаж': 3,
                    '1,2,3,4, подвал': 4,
                    '1,2,подвал ': 2,
                    '1-3 этажи, цоколь (188,4 кв.м), подвал (104 кв.м)': 3,
                    '1-7': 7,
                    '2, 3, 4, тех.этаж': 4,
                    '2-й': 2,
                    '3 этаж, мансарда (4 этаж)': 4,
                    '3, 4': 4,
                    '3,4': 4,
                    '5(мансарда)': 5,
                    'Техническое подполье': -1,
                    'подвал, 1 и 4 этаж': 4,
                    'подвал, 1, 2': 2,
                    'подвал, 1, 2, 3': 3,
                    'подвал, 2': 2,
                    'подвал,1,2,3': 3,
                    'технический этаж,5,6': 6,
                    'цоколь, 1, 2,3,4,5,6': 6,
                    'цокольный': 0,
                    'цокольный, 1,2':2
                    }

In [780]:
train_data = pd.read_csv('./data/data/train.csv')
test_data = pd.read_csv('./data/data/test.csv')
print(test_data.shape)

(2974, 76)


In [781]:
def is_float(value):
    try:
        if type(float(value)) != 'float':
            return True
    except:
        return False

In [782]:
train_data['day'] = pd.to_datetime(train_data['date']).dt.day
test_data['day'] = pd.to_datetime(test_data['date']).dt.day

train_data['floor_isnull'] = np.where(train_data['floor'].isnull(), 0, 1)
test_data['floor_isnull'] = np.where(test_data['floor'].isnull(), 0, 1)

train_data['floor_is_float'] = np.where(train_data['floor'].apply(lambda x: is_float(x)), 1, 0)
test_data['floor_is_float'] = np.where(test_data['floor'].apply(lambda x: is_float(x)), 1, 0)

print(test_data.shape)

(2974, 79)


In [783]:
train_data['address'] = train_data['city'] + ' ' + train_data['street']
test_data['address'] = test_data['city'] + ' ' + test_data['street']

train_data['log_total_square'] = np.log(train_data['total_square'])
test_data['log_total_square'] = np.log(test_data['total_square'])
        
train_data['clear_city'] = train_data['city'].apply(lambda x: x.split(', ')[-1].strip())
test_data['clear_city'] = test_data['city'].apply(lambda x: x.split(', ')[-1].strip())

print(test_data.shape)

(2974, 82)


In [784]:
def create_agg_osm_features(train_data, test_data=None):
    osm_cols = [col for col in list(train_data) if 'osm_' in col and 'name' not in col]
    
    for col in osm_cols:
        train_data[col] = train_data[col].astype('float64')
        agg_data = (train_data.groupby('clear_city')[col].sum() / train_data[col].sum() * 100).reset_index()
        if type(test_data) != type(None):
            test_data = pd.merge(test_data, agg_data, on='clear_city', how='left', suffixes=('', '_agg')) 
        train_data = pd.merge(train_data, agg_data, on='clear_city', how='left', suffixes=('', '_agg'))
        
    if type(test_data) != type(None):
        return train_data, test_data
    else:
        return train_data
    
print(test_data.shape)

(2974, 82)


In [785]:
train_data, test_data = create_agg_osm_features(train_data, test_data)

print(test_data.shape)

(2974, 138)


In [786]:
test_data.shape, train_data.shape

((2974, 138), (279792, 139))

In [787]:
add_data = pd.read_csv('./data/data/add_data.csv')

train_data['region'] = train_data['region'].apply(lambda x: x.lower().strip())
test_data['region'] = test_data['region'].apply(lambda x: x.lower().strip())

train_data = pd.merge(train_data, add_data, on='region', how='left', suffixes=('', ''))
test_data = pd.merge(test_data, add_data, on='region', how='left', suffixes=('', ''))

In [788]:
train_data['text_floor'] = np.where(train_data['floor'].isnull(), 
                                    'unknown_floor', 
                                    train_data['floor'].apply(lambda x: str(x)))

train_data.loc[train_data['floor_is_float'] == 0, 'floor'] = \
    train_data.loc[train_data['floor_is_float'] == 0, 'floor'].map(text_floor_replace)

test_data['text_floor'] = np.where(test_data['floor'].isnull(), 
                                    'unknown_floor', 
                                    test_data['floor'].apply(lambda x: str(x)))

test_data.loc[test_data['floor_is_float'] == 0, 'floor'] = \
    test_data.loc[test_data['floor_is_float'] == 0, 'floor'].map(text_floor_replace)

In [789]:
features_for_log = [
                   'osm_city_closest_dist',
                    'osm_train_stop_closest_dist',
                    'osm_transport_stop_closest_dist',
                    'osm_crossing_closest_dist',
                    'reform_mean_floor_count_500'
                   ]

def log_features(data, lst):
    for col in lst:
        name_log_col = '_'.join(['log', col])
        data[name_log_col] = np.log(data[col] + 0.00001)
    return data
        
train_data = log_features(train_data, features_for_log)
test_data = log_features(test_data, features_for_log)

In [790]:
def filter_outliers(data, cols_to_filter: list):

    for col in cols_to_filter:
        data = data[(data[col] < data[col].mean() + 3 * data[col].std()) & (data[col] > data[col].mean() - 3 * data[col].std())]
    
    return data

train_data = filter_outliers(train_data, ['reform_mean_year_building_1000', 'reform_mean_year_building_500'])

In [791]:
eur_rub = pd.read_csv('eur_rub.txt', sep=',')
usd_rub = pd.read_csv('usd_rub.txt', sep=',')

In [792]:
train_data = pd.merge(train_data, eur_rub, on='date', how='left')
test_data = pd.merge(test_data, eur_rub, on='date', how='left')

train_data = pd.merge(train_data, usd_rub, on='date', how='left')
test_data = pd.merge(test_data, usd_rub, on='date', how='left')

In [793]:
drop_cols = [
            #'rounded_lng',
            #'rounded_lat',
            'date', 
             'city',
             'street',
             #'id',
            'clear_city',
            ]

train_data.drop(drop_cols, axis=1, inplace=True)
test_data.drop(drop_cols, axis=1, inplace=True)

In [794]:
test_data.shape, train_data.shape

((2974, 170), (245445, 171))

In [795]:
cols_to_discretize = {
                    'osm_shops_points_in_0.001': 2,
                    'osm_shops_points_in_0.005': 2,
                    'osm_shops_points_in_0.0075': 2,
                    'osm_shops_points_in_0.01': 2,
                    'osm_finance_points_in_0.005': 2,
                    'osm_finance_points_in_0.0075': 2,
                    'osm_finance_points_in_0.01': 2,
                    'osm_leisure_points_in_0.005': 2,
                    'osm_leisure_points_in_0.01': 2,
                    'osm_amenity_points_in_0.01': 2,
                    'osm_transport_stop_points_in_0.005': 2,
                    'osm_crossing_points_in_0.005': 2,
                    'osm_crossing_points_in_0.0075': 2,
                    'osm_crossing_points_in_0.01': 2,
                    'reform_count_of_houses_1000': 2,
                    'reform_count_of_houses_500': 2,
                    'reform_mean_year_building_500': 2,
                    'reform_mean_year_building_1000': 2,
                    'osm_transport_stop_points_in_0.0075': 3,
                    'osm_transport_stop_points_in_0.01': 3
                }

def bins_discretize_data(train_data, test_data, columns):
    data = pd.concat([train_data, test_data])
    for col, n_bins in columns.items():
        if data[col].isnull().sum() == 0:
            est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
            discr_col = est.fit_transform(np.array(data[col]).reshape(-1, 1))
            name_col = 'bin_' + col
            data[name_col] = discr_col
    return data

In [796]:
data = bins_discretize_data(train_data, test_data, cols_to_discretize)

In [797]:
train_data = data[~data['per_square_meter_price'].isnull()]
test_data = data[data['per_square_meter_price'].isnull()]

train_data.shape, test_data.shape

((245445, 189), (2974, 189))

In [798]:
train_data.to_csv('./preprocessed_data/train_data.csv', index=False)
test_data.to_csv('./preprocessed_data/test_data.csv', index=False)

In [ ]:
# train_data = pd.get_dummies(train_data, columns=['region'])
# test_data = pd.get_dummies(test_data, columns=['region'])

# region_cols = [col for col in list(train_data) if 'region' in col]

# for col in region_cols:
#     if col not in list(test_data):
#         test_data[col] = np.zeros(test_data.shape[0])

# def geo(lat_lng):
#     geolocator = Nominatim(user_agent="user_agent geopy/2.2.0")
#     adr = geolocator.reverse(lat_lng).raw['address']
#     try:
#         return adr['road']
#     except:
#         return np.nan

# def find_road(data):
#     data['lat_lng'] = data['lat'].apply(lambda x: str(x)) + ', ' + data['lng'].apply(lambda x: str(x))
#     data['road'] = data['lat_lng'].apply(lambda x: geo(x))
#     return data

# train_data = find_road(train_data)
# test_data = find_road(test_data)

# def fill_na_grouped_data(data, col_to_fill: str, group_cols: list, train_mode=True):
#         name_gr = '_'.join(group_cols)
#         fill_data_folder_name = 'fill_data'
#         fill_data_path = os.path.join('./', fill_data_folder_name)
#         fill_data_filename = f'{col_to_fill}_{name_gr}.csv'
#         if train_mode:
#             grouped_data = data.groupby(group_cols)[col_to_fill].median()
#             grouped_data.dropna(inplace=True)
#             grouped_data.to_csv(os.path.join('./', fill_data_folder_name, fill_data_filename))
#         else:
#             grouped_data = pd.read_csv(os.path.join('./', fill_data_folder_name, fill_data_filename),
#                                        index_col=group_cols)
#         grouped_data = grouped_data.squeeze(axis=0 if train_mode else 1)
#         data.set_index(group_cols, inplace=True)
#         data[col_to_fill].fillna(grouped_data, inplace=True)
#         data.reset_index(inplace=True)
#         return data

# train_data['rounded_lng'] = train_data['lng'].apply(lambda x: round(x, 1))
# train_data['rounded_lat'] = train_data['lat'].apply(lambda x: round(x, 1))

# test_data['rounded_lng'] = test_data['lng'].apply(lambda x: round(x, 1))
# test_data['rounded_lat'] = test_data['lat'].apply(lambda x: round(x, 1))

# train_data = fill_na_grouped_data(train_data, 'reform_mean_floor_count_500', ['rounded_lng', 'rounded_lat'])
# train_data = fill_na_grouped_data(train_data, 'reform_mean_year_building_500', ['rounded_lng', 'rounded_lat'])
# train_data = fill_na_grouped_data(train_data, 'reform_house_population_500', ['rounded_lng', 'rounded_lat'])
# train_data = fill_na_grouped_data(train_data, 'reform_mean_floor_count_1000', ['rounded_lng', 'rounded_lat'])
# train_data = fill_na_grouped_data(train_data, 'reform_mean_year_building_1000', ['rounded_lng', 'rounded_lat'])
# train_data = fill_na_grouped_data(train_data, 'reform_house_population_1000', ['rounded_lng', 'rounded_lat'])

# test_data = fill_na_grouped_data(test_data, 'reform_mean_floor_count_500', ['rounded_lng', 'rounded_lat'], False)
# test_data = fill_na_grouped_data(test_data, 'reform_mean_year_building_500', ['rounded_lng', 'rounded_lat'], False)
# test_data = fill_na_grouped_data(test_data, 'reform_house_population_500', ['rounded_lng', 'rounded_lat'], False)
# test_data = fill_na_grouped_data(test_data, 'reform_mean_floor_count_1000', ['rounded_lng', 'rounded_lat'], False)
# test_data = fill_na_grouped_data(test_data, 'reform_mean_year_building_1000', ['rounded_lng', 'rounded_lat'], False)
# test_data = fill_na_grouped_data(test_data, 'reform_house_population_1000', ['rounded_lng', 'rounded_lat'], False)